In [1]:
from google.colab import files
data = files.upload()

Saving crime data2.csv to crime data2 (4).csv


In [2]:
!pip install scipy==1.2

In [0]:
import io
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

C2 = pd.read_csv(io.BytesIO(data['crime data2.csv']))

In [4]:
C2.dtypes

City                    object
Population              object
Violent crime           object
Murder                 float64
Rape revised            object
Rape legacy            float64
Robbery                 object
Aggravated assault      object
Property crime          object
Burglary                object
Larceny                 object
Motor Vehicle theft     object
Arson                  float64
dtype: object

In [5]:
C2.head()

,City,Population,Violent crime,Murder,Rape revised,Rape legacy,Robbery,Aggravated assault,Property crime,Burglary,Larceny,Motor Vehicle theft,Arson
0,Adams Village,"1,851",0,0.0,NaN,0.0,0,0,11,1,10,0,0.0
1,Addison Town and Village,"2,568",2,0.0,NaN,0.0,1,1,49,1,47,1,0.0
2,Afton Village4,820,0,0.0,0,NaN,0,0,1,0,1,0,0.0
3,Akron Village,"2,842",1,0.0,NaN,0.0,0,1,17,0,17,0,0.0
4,Albany4,"98,595",802,8.0,54,NaN,237,503,"3,888",683,"3,083",122,12.0


In [6]:
C2.columns = C2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
C2.head()

,city,population,violent_crime,murder,rape_revised,rape_legacy,robbery,aggravated_assault,property_crime,burglary,larceny,motor_vehicle_theft,arson
0,Adams Village,"1,851",0,0.0,NaN,0.0,0,0,11,1,10,0,0.0
1,Addison Town and Village,"2,568",2,0.0,NaN,0.0,1,1,49,1,47,1,0.0
2,Afton Village4,820,0,0.0,0,NaN,0,0,1,0,1,0,0.0
3,Akron Village,"2,842",1,0.0,NaN,0.0,0,1,17,0,17,0,0.0
4,Albany4,"98,595",802,8.0,54,NaN,237,503,"3,888",683,"3,083",122,12.0


In [0]:
#converting to strongs so I can remove commas

C2['robbery'] = C2['robbery'].astype(str)
C2['population'] = C2['population'].astype(str)
C2['violent_crime'] = C2['violent_crime'].astype(str)
C2['property_crime'] = C2['property_crime'].astype(str)
C2['burglary'] = C2['burglary'].astype(str)
C2['larceny'] = C2['larceny'].astype(str)
C2['motor_vehicle_theft'] = C2['motor_vehicle_theft'].astype(str)

#recasting as floats so they can discretized 
C2['robbery'] = C2['robbery'].str.replace("," , "").astype(float) #Removing commas and changing dtype to float otherwise I cannot discretize Robbery
C2['population'] = C2['population'].str.replace("," , "").astype(float)
C2['violent_crime'] = C2['violent_crime'].str.replace("," , "").astype(float)
C2['property_crime'] = C2['property_crime'].str.replace("," , "").astype(float)
C2['burglary'] = C2['burglary'].str.replace("," , "").astype(float)
C2['larceny'] = C2['larceny'].str.replace("," , "").astype(float)
C2['motor_vehicle_theft'] = C2['motor_vehicle_theft'].str.replace("," , "").astype(float)



In [8]:
C2.head(2)

,city,population,violent_crime,murder,rape_revised,rape_legacy,robbery,aggravated_assault,property_crime,burglary,larceny,motor_vehicle_theft,arson
0,Adams Village,1851.0,0.0,0.0,NaN,0.0,0.0,0,11.0,1.0,10.0,0.0,0.0
1,Addison Town and Village,2568.0,2.0,0.0,NaN,0.0,1.0,1,49.0,1.0,47.0,1.0,0.0


In [9]:
# discretizing 
C2F = pd.get_dummies(C2['city'])
C2F['murder'] = np.where(C2['murder']>0, 1, 0)
C2F['robbery'] = np.where(C2['robbery']>0, 1, 0)
C2F['burglary'] = np.where(C2['burglary']>0, 1, 0)
C2F['motor_vehicle_theft'] = np.where(C2['motor_vehicle_theft']>0, 1, 0)
C2F['arson'] = np.where(C2['arson']>0, 1, 0)
C2F['larceny'] = np.where(C2['larceny']>0, 1,0)


C2F.head()

,,Adams Village,Addison Town and Village,Afton Village4,Akron Village,Albany4,Albion Village4,Alexandria Bay Village4,Alfred Village4,Altamont Village4,Amherst Town4,Amity Town and Belmont Village4,Amityville Village,Amsterdam,Angelica Village4,Arcade Village,Ardsley Village,Asharoken Village,Attica Village,Avon Village4,Baldwinsville Village4,Ballston Spa Village4,Batavia4,Bath Village4,Beacon4,Bedford Town4,Bethlehem Town4,Binghamton4,Black River,Blooming Grove Town4,Bolton Town,Boonville Village,Brant Town,Brewster,Briarcliff Manor Village,Brighton Town4,Brockport Village4,Bronxville Village,Brownville Village,Buffalo4,...,Tuxedo Park Village,Tuxedo Town4,Ulster Town4,Utica4,Vernon Village,Vestal Town4,Walden Village4,Wallkill Town4,Walton Village4,Warsaw Village4,Washingtonville Village,Waterford Town and Village,Waterloo Village4,Watertown4,Watervliet4,Watkins Glen Village4,Waverly Village4,Webster Town and Village4,Weedsport Village,Wellsville Village,West Seneca Town4,Westfield Village4,Westhampton Beach Village,White Plains4,Whitehall Village4,Whitesboro Village,Whitestown Town,Windham Town4,Wolcott Village,"Woodbury Town4, 5",Woodstock Town4,Yonkers4,Yorktown Town4,Youngstown Village4,murder,robbery,burglary,motor_vehicle_theft,arson,larceny
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1


In [10]:
import math
import warnings

from IPython.display import display
from sklearn import linear_model
import statsmodels.formula.api as smf

C2F['logged_prop_crime'] = np.log(1+ C2['property_crime'])
C2F['population'] = C2['population']
C2F['pop2'] = (C2F.population)**2

#Reminder my formula is Property Crime ~ population+population^2+robbery+burglary+motor_vehicle_theft+arson

C2F.head()

,,Adams Village,Addison Town and Village,Afton Village4,Akron Village,Albany4,Albion Village4,Alexandria Bay Village4,Alfred Village4,Altamont Village4,Amherst Town4,Amity Town and Belmont Village4,Amityville Village,Amsterdam,Angelica Village4,Arcade Village,Ardsley Village,Asharoken Village,Attica Village,Avon Village4,Baldwinsville Village4,Ballston Spa Village4,Batavia4,Bath Village4,Beacon4,Bedford Town4,Bethlehem Town4,Binghamton4,Black River,Blooming Grove Town4,Bolton Town,Boonville Village,Brant Town,Brewster,Briarcliff Manor Village,Brighton Town4,Brockport Village4,Bronxville Village,Brownville Village,Buffalo4,...,Utica4,Vernon Village,Vestal Town4,Walden Village4,Wallkill Town4,Walton Village4,Warsaw Village4,Washingtonville Village,Waterford Town and Village,Waterloo Village4,Watertown4,Watervliet4,Watkins Glen Village4,Waverly Village4,Webster Town and Village4,Weedsport Village,Wellsville Village,West Seneca Town4,Westfield Village4,Westhampton Beach Village,White Plains4,Whitehall Village4,Whitesboro Village,Whitestown Town,Windham Town4,Wolcott Village,"Woodbury Town4, 5",Woodstock Town4,Yonkers4,Yorktown Town4,Youngstown Village4,murder,robbery,burglary,motor_vehicle_theft,arson,larceny,logged_prop_crime,population,pop2
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2.484907,1851.0,3.426201e+06
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,3.912023,2568.0,6.594624e+06
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.693147,820.0,6.724000e+05
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.890372,2842.0,8.076964e+06
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,8.265907,98595.0,9.720974e+09


In [11]:
reg1 = linear_model.LinearRegression()
Y = C2F['logged_prop_crime'].fillna(0).values.reshape(-1, 1)
X = C2F[['population' , 'pop2', 'robbery', 'burglary' , 'arson' , 'larceny' , 'motor_vehicle_theft']].fillna(0) #attempting to fill NaN values 
reg1.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', reg1.coef_)
print('\nIntercept: \n', reg1.intercept_)
print('\nR-squared:')
print(reg1.score(X, Y))


Coefficients: 
 [[ 2.46197452e-05 -2.80622206e-12  9.22074625e-01  1.20452174e+00
   6.17028610e-01  1.90533823e+00  9.10201015e-01]]

Intercept: 
 [-0.24451931]

R-squared:
0.8451511827593403


In [0]:
# Write out the model formula.
# Your dependent variable on the right, independent variables on the left
# Use a ~ to represent an '=' from the functional form
linear_formula = 'logged_prop_crime ~ robbery+burglary+motor_vehicle_theft+arson+larceny+population+pop2'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=C2F).fit()

In [13]:
lm.params

Intercept             -2.272447e-01
robbery                9.509294e-01
burglary               1.270169e+00
motor_vehicle_theft    8.862968e-01
arson                  6.058021e-01
larceny                1.833319e+00
population             2.458368e-05
pop2                  -2.802158e-12
dtype: float64

In [14]:
lm.pvalues

Intercept              3.513464e-01
robbery                1.458598e-19
burglary               6.389676e-17
motor_vehicle_theft    5.684920e-16
arson                  1.462178e-09
larceny                3.334365e-12
population             1.865628e-38
pop2                   1.663764e-36
dtype: float64

In [15]:
lm.rsquared

0.8299173059208734

In [16]:
lm.conf_int()

,0,1
Intercept,-7.061322e-01,2.516428e-01
robbery,7.560224e-01,1.145836e+00
burglary,9.858894e-01,1.554449e+00
motor_vehicle_theft,6.808813e-01,1.091712e+00
arson,4.139557e-01,7.976485e-01
larceny,1.333220e+00,2.333419e+00
population,2.128420e-05,2.788316e-05
pop2,-3.191234e-12,-2.413081e-12


In [17]:
from sklearn.feature_selection import f_regression, mutual_info_regression

Ftest = f_regression(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
Ftest

(array([ 23.38198989,  16.72083044, 311.99499766, 165.31276681,
        121.15743254, 117.7679438 , 275.64003045]),
 array([1.94356211e-06, 5.30449720e-05, 3.31054132e-51, 1.38734484e-31,
        1.34093314e-24, 4.89571003e-24, 9.05744952e-47]))

In [19]:
C2F.head()

,,Adams Village,Addison Town and Village,Afton Village4,Akron Village,Albany4,Albion Village4,Alexandria Bay Village4,Alfred Village4,Altamont Village4,Amherst Town4,Amity Town and Belmont Village4,Amityville Village,Amsterdam,Angelica Village4,Arcade Village,Ardsley Village,Asharoken Village,Attica Village,Avon Village4,Baldwinsville Village4,Ballston Spa Village4,Batavia4,Bath Village4,Beacon4,Bedford Town4,Bethlehem Town4,Binghamton4,Black River,Blooming Grove Town4,Bolton Town,Boonville Village,Brant Town,Brewster,Briarcliff Manor Village,Brighton Town4,Brockport Village4,Bronxville Village,Brownville Village,Buffalo4,...,Utica4,Vernon Village,Vestal Town4,Walden Village4,Wallkill Town4,Walton Village4,Warsaw Village4,Washingtonville Village,Waterford Town and Village,Waterloo Village4,Watertown4,Watervliet4,Watkins Glen Village4,Waverly Village4,Webster Town and Village4,Weedsport Village,Wellsville Village,West Seneca Town4,Westfield Village4,Westhampton Beach Village,White Plains4,Whitehall Village4,Whitesboro Village,Whitestown Town,Windham Town4,Wolcott Village,"Woodbury Town4, 5",Woodstock Town4,Yonkers4,Yorktown Town4,Youngstown Village4,murder,robbery,burglary,motor_vehicle_theft,arson,larceny,logged_prop_crime,population,pop2
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2.484907,1851.0,3.426201e+06
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,3.912023,2568.0,6.594624e+06
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.693147,820.0,6.724000e+05
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.890372,2842.0,8.076964e+06
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,8.265907,98595.0,9.720974e+09


In [20]:
#X= C2F[['population' , 'pop2', 'robbery', 'burglary' , 'arson' , 'larceny' , 'motor_vehicle_theft']]
from sklearn.decomposition import PCA
pca = PCA(n_components=7)
pca.fit(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component \n',
    pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component 
 [1.00000000e+00 4.87364591e-17 2.58220285e-26 1.06179370e-26
 8.00287382e-27 6.09341016e-27 2.36191488e-27]


In [24]:
# Test your model with different holdout groups.
#reg1.fit(X, Y)
#print('\nCoefficients: \n', reg1.coef_)
#print('\nIntercept: \n', reg1.intercept_)
#print('\nR-squared:')
#print(reg1.score(X, Y))

from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

reg2 = linear_model.LinearRegression()

holdout = reg2.fit(X_train, y_train)
sample = reg2.fit(X, Y)


print('With 20% Holdout: ' + str(reg2.score(X_train, y_train)))
print('Testing on Sample: ' + str(reg2.score(X,Y)))

With 20% Holdout: 0.8367747178083861
Testing on Sample: 0.8451511827593403


In [25]:
# Cross validation

from sklearn.model_selection import cross_val_score
cross_val_score(reg2, X, Y, cv=10)

array([ 8.42776242e-01,  7.97806859e-01,  5.95470983e-01,  7.41979466e-01,
        8.45781671e-01, -6.91698829e+05,  8.20299161e-01,  8.77078694e-01,
        9.06147819e-01,  8.60088481e-01])

In [27]:
# Adding Violent Crime and Murder to my model
C2F['violent_crime'] = np.where(C2['violent_crime']>0, 1,0)


reg3 = linear_model.LinearRegression()

#Making a new feature df for the new model
X2 = C2F[['population' , 'pop2', 'robbery', 'burglary' , 'arson' , 'larceny' , 'motor_vehicle_theft' , 'murder' , 'violent_crime']].fillna(0) #attempting to fill NaN values 
reg3.fit(X2, Y)

# Inspect the results.
print('\nCoefficients: \n', reg3.coef_)
print('\nIntercept: \n', reg3.intercept_)
print('\nR-squared:')
print(reg3.score(X2, Y))


Coefficients: 
 [[ 2.07071150e-05 -2.35273768e-12  7.96153181e-01  1.08935666e+00
   5.41406041e-01  1.81213017e+00  9.01904695e-01  6.10566283e-01
   3.14665801e-01]]

Intercept: 
 [-0.23489235]

R-squared:
0.8559225397330142


In [28]:
cross_val_score(reg3, X2, Y, cv=10)

array([ 8.59219283e-01,  7.87100961e-01,  6.07166175e-01,  7.62436472e-01,
        8.64258296e-01, -5.97740563e+05,  8.18697518e-01,  8.83572227e-01,
        9.10639680e-01,  8.60016006e-01])